# Module INM378/IN3031: Digital Signal Processing and Audio Programming
## Lab 6 - Game Audio and Real-Time DSP in Python and FMOD

### 0. Setup dependencies

In [ ]:
try:
    import google.colab
    import subprocess
    import os.path
    p = subprocess.run(['git', 'rev-parse', '--is-inside-work-tree'], stdout=subprocess.PIPE, universal_newlines=True)
    if p.stdout == 'true\n':
        !git pull
    else:
        if not os.path.isdir('city_dsp_ap'):
            !git clone --depth 1 -q https://github.com/jpauwels/city_dsp_ap.git
        %cd city_dsp_ap
except:
    %cd city_dsp_ap

In [ ]:
import numpy as np
from scipy.io import wavfile
from dataclasses import dataclass
from dsp_ap.util import Audio

### 1. Introduction

This week we'll have a look at how to apply the previously DSP concepts in real-time. This is particularly relevant for game audio or audio workstation plugins. Since C++ is the dominant language in both industries, we'll also attempt to make the leap from Python to C++. We'll implement a real-time DSP effect in the FMOD audio engine by modifying the `CustomDSP.zip` template available on Moodle. To make this conceptually easier, a highly simplified version of some FMOD functions has been implemented in this notebook in Python.

If you know the basics of C++, you are strongly encouraged to try implementing today's lab in FMOD, even if you are taking the data stream for the coursework and have never worked with game technologies. The template only requires modification of the DSP code, so you don't need any prior knowledge about games or 3D modelling.

If you've never seen C++ code before, there's no need to learn it just for this session :-) Implementing the answers to the questions in this notebook will give you the same conceptual understanding as you'd get from implementing them in FMOD.

### 2. Custom DSP in FMOD

For (near) real-time audio, we don't read an audio file in its entirety, but rather process it in a streaming fashion. We get the audio piece-by-piece as input buffers of a certain block size, and are expected to write processed output to output buffers of the same size. Crucial is that we don't control the size of these input blocks ourselves, they are set by environment (in this case FMOD) and can vary between different runs (or even change during execution).

The smaller this buffer size is, the faster the output corresponding to the input can be played back to the user, so the less latency. On the other hand, the reading and processing of the input audio can be interrupted by other processes happening at the same time, so in order to guarantee a consistent stream of output, it is necessary to buffer the input to a certain extent. The ideal buffer size is therefore needs to strike the right balance between as small as possible such that latency can be reduced as much as possible, but not so small that the output stream gets interrupted (audible as "crackling" of the sound).

For this task, we're going to transplant the functionality of the FMOD demo available at `api/core/examples/dsp_custom.cpp` in the [FMOD Studio API](https://www.fmod.com/download) into the OpenGL template `CustomDSP` from Moodle (which already has a basic FMOD configuration). This will demonstrate how to write custom DSP functions as callbacks in FMOD and how to share data between multiple invocations of the callback.

```
FMOD_RESULT F_CALLBACK myDSPCallback(FMOD_DSP_STATE* dsp_state, float* inbuffer, float* outbuffer, 
                                     unsigned int length, int inchannels, int* outchannels) 
{
    mydsp_data_t *data = (mydsp_data_t *)dsp_state->plugindata;
    
    for (unsigned int samp = 0; samp < length; samp++) 
    {
        for (int chan = 0; chan < *outchannels; chan++)
        {
            data->buffer[(samp * *outchannels) + chan] = 
                inbuffer[(samp * inchannels) + chan] * data->volume_linear;
            outbuffer[(samp * inchannels) + chan] = data->buffer[(samp * *outchannels) + chan];
        }
    }
    return FMOD_OK; 
} 
```

A simplified version of the callback function in `dsp_custom.cpp` is printed above. It's in this function that the actual DSP code needs to be implemented. We can see that at every invocation, we get an `inbuffer` of size `length` and are expected to write our output to an `outbuffer` of the same size. These buffers can contain multiple channels, which are stored interleaved. For instance, a stereo signal containing a left channel `[l1, l2, l3]` and a right channel `[r1, r2, r3]` will come in as an `inbuffer` containing `[l1, r1, l2, r2, l3, r3]`. The `length` of this `inbuffer` will be $3$, meaning that it gives the size of the input *per channel*. Finally, the `dsp_state` is the object that allows to share information between multiple function invocations.

The Python equivalent of this function is given below.

In [ ]:
def myDSPCallback(data, inbuffer, outbuffer, length, inchannels, outchannels):
    for samp in range(length):
        for chan in range(outchannels):
            data.buffer[samp*outchannels+chan] = inbuffer[samp*inchannels+chan] * data.volume_linear
            outbuffer[samp*inchannels+chan] = data.buffer[samp*outchannels+chan]

Both the C++ and the Python function assume that the number of input channels is equal to the number of output channels. It is a reasonable assumption for both this lab and the coursework. We can ensure this in the creation of the DSP plugin (the comment in `dsp_custom.cpp` tells how to do this in FMOD and it's also done in the Python code below).

The shared data in FMOD is a custom struct of datatype `mydsp_data_t` that has the following form:
```
typedef struct 
{
    float volume_linear;
    float* buffer;
} mydsp_data_t;
```

In the Python code, its usage is slightly simplified by passing the data object directly to the function, without passing through `FMOD_DSP_STATE` but the actual data structure is similarly defined below.

In [ ]:
@dataclass
class DSP_Data:
    volume_linear: float
    buffer: np.array

We see that it contains two fields `volume_linear` and `buffer`, which are set in Python by specifying their values to the datatype `DSP_Data` between round brackets. We can then access their values with a dot-syntax like `data.volume_linear`, just like the C++ struct.

In [ ]:
data = DSP_Data(1, np.zeros(5))
print('Shared data structure: ', data)
print('Volume parameter: ', data.volume_linear)
print('Buffer persisting between callbacks: ', data.buffer)

In the callback, we iterate through the samples in the input buffer and rescale them using the value of `volume_linear`, stored in the shared data object. We store the result in a persistent buffer (not to be confused with either input or output buffer). The output values are read back from this buffer.

Note that it is not necessary in this simple case to use a buffer to store the rescaled values. The same functionality can be achieved by passing the input directly to the output. The buffer is just used here as an example of how you can pass sample data between consecutive function calls. Moreover, the size of the persistent buffer needs to be set manually. In this case, it needs to be (at least) as big as the block size (so not $5$ as configured above), but this depends on the buffer's use. For instance, to implement a delay, it would need to be as large as the (maximal) number of samples the signal gets delayed. Don't forget that it also needs to accommodate multiple channels if necessary.

To test our callback function, we can create a mock real-time environment, where the samples of a signal get passed into the callback as multiple blocks. The code for this is provided below, but you don't need to understand exactly what's going on in there, you're just going to run it later.

In [ ]:
def run_realtime(input_samples, block_size, callback_fn, state_data):
    num_channels = input_samples.shape[0] if input_samples.ndim > 1 else 1
    input_stream = input_samples.T.ravel()
    total_length = len(input_stream)
    output_stream = np.empty(total_length, dtype=input_stream.dtype)

    # Pass input sliced into blocks to callback function
    for start in np.arange(0, total_length - block_size*num_channels, block_size*num_channels):
        input_buffer = input_stream[start:start+block_size*num_channels]
        output_buffer = output_stream[start:start+block_size*num_channels]
        callback_fn(state_data, input_buffer, output_buffer, block_size, num_channels, num_channels)

    # Pass partial buffer at end of signal
    start += block_size*num_channels
    remaining_length = (total_length - start) // num_channels
    callback_fn(state_data, input_stream[start:], output_stream[start:], remaining_length, num_channels, num_channels)

    output_samples = output_stream.reshape(-1, num_channels).T if num_channels > 1 else output_stream
    return output_samples

We can then read an audio file to pass to the callback function.

In [ ]:
samplerate, input_samples = wavfile.read('audio/rockA.wav')
input_samples = input_samples.T # stereo playback requires transposition of columns and rows
num_channels = input_samples.shape[0] if input_samples.ndim > 1 else 1

# Play back input
print('Input')
display(Audio(input_samples, rate=samplerate))

The callback should be working for any given block size and its output should be exactly the same regardless of the block size, so we test a couple of values.

In [ ]:
for block_size in [256, 512, 1024]:
    data = DSP_Data(0.5, np.zeros(block_size * num_channels))
    output_samples = run_realtime(input_samples, block_size, myDSPCallback, data)
    print('Output for block size {}'.format(block_size))
    display(Audio(output_samples, rate=samplerate))

The shared data in Python is created by a simple assignment of the datatype constructor. In FMOD, this is done using another callback function, called `myDSPCreateCallback`. When you dynamically allocate memory in `myDSPCreateCallback`, also implement a matching `myDSPReleaseCallback` to release it afterwards.

Using another callback (with an added layer of indirection through `FMOD_DSP_STATE`) allows to modify the shared state externally. For instance, this makes it possible to map the volume dynamically to another control, such as the position of the camera. Such functionality is not possible in this simple Python version, and not necessary for this lab, but will be required for the coursework. Callbacks such as `myDSPSetParameterFloatCallback` can be used to set the parameter values externally. Check `dsp_custom.cpp` for an example of its usage.

Now that we have a working custom DSP callback function, implement in Python and/or FMOD the following two exercises:
1. Create a circular buffer with 4096 samples per channel and use it to implement the maximally possible delay. Rescale the current input signal and the delayed input signal in equal proportion to avoid clipping the signal. 
2. Create a simple averaging FIR filter with 4 coefficients in the time domain. Use the minimum size circular buffer to make it work.

In order to solve these exercises, you'll have to modify the shared data object and the custom DSP callback function. Note that the modulo operator `%` comes before additions and subtractions in the order of operations, so if you want to take the modulo of an expression containing a `+` or `-`, remember to surround it by parentheses.

In [ ]:
# write your code here

In [ ]:
# write your code here

### 3. Additional task for games students – 3D Audio

Get the `3DAudio.zip` template from Moodle and modify it as follows:

1. Set the volume on the music channel to be lower by default.
2. Change the music volume with `+ (VK_ADD)` and `- (VK_SUBTRACT)` key presses.
3. Set the FMOD 3D settings to some sensible values.
4. Refactor the "event sound" (triggered with "1") to play through a 3D channel.
5. Set the camera as the FMOD listener, and update the position as the camera moves.
6. Set the event sound to come from the horse's position.
7. Play around with the FMOD 3D settings to get a sense of their effect.

If you got this far, you should be familiar with all the FMOD functionality needed to complete the games stream coursework.